<a href="https://colab.research.google.com/github/blancavazquez/CursoDatosMasivos/blob/master/notebooks/3c_minhash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsqueda de documentos con MinHash
En esta libreta veremos cómo hacer búsqueda eficiente de documentos similares considerando la similitud de Jaccard usando MinHash.

Primero cargamos los módulos necesarios.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from math import floor 

import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

Vamos a usar el conjunto de documentos de _20 Newsgropus_, el cual descargamos usando scikit-learn. 

In [0]:
db = fetch_20newsgroups(remove=('headers','footers','quotes'))

Definimos nuestro analizador léxico usando la biblioteca NLTK. Vamos a extraer los componentes léxicos, pasarlos a minúsculas y lematizarlos.

In [3]:
import nltk
nltk.download(['punkt','averaged_perceptron_tagger','wordnet'])
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.corpus.reader.wordnet import NOUN, VERB, ADV, ADJ

morphy_tag = {
    'JJ' : ADJ,
    'JJR' : ADJ,
    'JJS' : ADJ,
    'VB' : VERB,
    'VBD' : VERB,
    'VBG' : VERB,
    'VBN' : VERB,
    'VBP' : VERB,
    'VBZ' : VERB,
    'RB' : ADV,
    'RBR' : ADV,
    'RBS' : ADV
}

def doc_a_tokens(doc):
  tagged = pos_tag(word_tokenize(doc.lower()))
  lemmatizer = WordNetLemmatizer()
  tokens = []
  for p,t in tagged:
    tokens.append(lemmatizer.lemmatize(p, pos=morphy_tag.get(t, NOUN)))

  return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Dividimos nuestro conjunto en 2 subconjuntos: los documentos de la base que se buscarán y los documentos de consulta

In [0]:
n = len(db.data)
perm = np.random.permutation(n).astype(int)
n_ej = int(floor(n * 0.95))

base = [db.data[i] for i in perm[:n_ej]]
consultas = [db.data[i] for i in perm[n_ej:]]

Calculamos las bolsas de palabras del conjunto base usando la clase `CountVectorizer` de scikit-learn.

In [0]:
docs_base = []
for d in base:
  d = d.replace('\n',' ').replace('\r',' ').replace('\t',' ')
  tokens = doc_a_tokens(d)
  docs_base.append(' '.join(tokens))
v = CountVectorizer(stop_words='english', max_features=5000, max_df=0.8)
bolsas_base = v.fit_transform(docs_base)

dim = bolsas_base.shape[1]

También calculamos las bolsas para las consultas.

In [0]:
docs_consultas = []
for d in consultas:
  d = d.replace('\n',' ').replace('\r',' ').replace('\t',' ')
  tokens = doc_a_tokens(d)
  docs_consultas.append(' '.join(tokens))

bolsas_consultas = v.transform(docs_consultas)

Finalmente, definimos nuestra clase para MinHash, la cual encapsula las funciones para calcular los valores MinHash, las tuplas y los índices, la tabla y las operaciones de inserción, búsqueda y eliminación sobre esta. 

In [0]:
class MinHashTable:
  def __init__(self, n_cubetas, t_tupla, dim):
    self.n_cubetas = n_cubetas
    self.tabla = [[] for i in range(n_cubetas)]
    self.dim = dim
    self.t_tupla = t_tupla
    self.perm = np.random.randint(0, np.iinfo(np.int32).max, size=(self.dim, self.t_tupla))
    self.rind = np.random.randint(0, np.iinfo(np.int32).max, size=(self.dim, self.t_tupla))
    self.a = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.b = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.primo = 4294967291

  def __repr__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas)]
    return "<TablaHash :%s >" % ('\n'.join(contenido))

  def __str__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas) if self.tabla[i]]
    return '\n'.join(contenido)

  def sl(self, x, i):
    return (self.h(x) + i) % self.n_cubetas

  def h(self, x):
    return x % self.primo

  def minhash(self, x):
    xp = self.perm[x]
    xi = self.rind[x]
    amin = xp.argmin(axis = 0)
    
    pmin = xp[amin, np.arange(0, self.t_tupla)]
    emin = xi[amin, np.arange(0, self.t_tupla)]

    return np.sum(self.a * pmin, dtype=np.ulonglong), np.sum(self.b * emin, dtype=np.ulonglong)
     
  def insertar(self, x, ident):
    mh, v2 = self.minhash(x)
  
    llena = True
    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        self.tabla[cubeta].append(mh)
        self.tabla[cubeta].append([ident])
        llena = False
        break
      elif self.tabla[cubeta][0] == mh:
        self.tabla[cubeta][1].append(ident)
        llena = False
        break

    if llena:
      print('¡Error, tabla llena!')

  def buscar(self, x):
    mh, v2 = self.minhash(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        return []
      elif self.tabla[cubeta][0] == mh:
        return self.tabla[cubeta][1]
        
    return []

  def eliminar(self, x, ident):
    mh, v2 = self.minhash(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        break
      elif self.tabla[cubeta][0] == mh:
        return self.tabla[cubeta][1].remove(ident)

    return -1

Ahora instanciamos esta clase tantas veces como tablas queramos para realizar la búsqueda.

In [0]:
n_tablas = 10
tablas = [MinHashTable(2**21, 3, dim) for _ in range(n_tablas)]

Definimos una función para convertir de matriz dispersa tipo CSR a una lista de listas. Nota que no se están considerando las frecuencias de las bolsas, por lo que la representación del documento es un conjunto.

In [0]:
def csr_to_ldb(csr):
  ldb = [[] for _ in range(csr.shape[0])]
  coo = csr.tocoo()    
  for i,j,v in zip(coo.row, coo.col, coo.data):
    ldb[i].append(j)

  return ldb

In [0]:
ll_base = csr_to_ldb(bolsas_base)
for j,l in enumerate(ll_base):
    if l:
      for i in range(n_tablas):
        tablas[i].insertar(l, j)

Recuperamos los documentos similares a nuestros documentos de consulta usando las tablas MinHash.

In [0]:
ll_consultas = csr_to_ldb(bolsas_consultas)
docs = []
for j,l in enumerate(ll_consultas):
  dc = []
  if l:
    for i in range(n_tablas):
      dc.extend(tablas[i].buscar(l))
  docs.append(set(dc))

Finalmente, calculamos la similitud Jaccard de los documentos recuperados con los de consulta y los ordenamos por similitud.



In [0]:
def similitud_jaccard(x, y):
  x = x.toarray()[0]
  y = y.toarray()[0]
  inter = np.count_nonzero(x * y)
  return inter / (np.count_nonzero(x) + np.count_nonzero(y) - inter)

def fuerza_bruta(ds, qs, fd):
  medidas = np.zeros(ds.shape[0])
  for i,x in enumerate(ds):
    medidas[i] = fd(qs, x)

  return np.sort(medidas)[::-1], np.argsort(medidas)[::-1]

sims = []
orden = []
for i,q in enumerate(bolsas_consultas):
  ld = list(docs[i])
  if ld:
    s,o = fuerza_bruta(bolsas_base[ld], q, similitud_jaccard)
    sims.append(s)
    orden.append([ld[e] for e in o])
  else:
    sims.append([])
    orden.append([])

In [48]:
print(sims[21])
print(orden[21])

[0.9122807  0.36619718 0.10655738 0.08571429 0.07462687 0.06603774
 0.06569343 0.05487805 0.05263158 0.04477612 0.04424779 0.04347826
 0.04054054]
[6726, 2789, 8135, 2301, 852, 3992, 4962, 8978, 8818, 1041, 6941, 4893, 9563]


Examinamos los documentos más similares a uno de los de consulta.

In [49]:
print(consultas[21])

#
#I'm trying to write some code that lets me draw lines and do rubber-band
#boxes in Motif/X.  I'm running on an 8-bit display for which I've created a
#colormap and am using almost all of the colors.  I want to draw the lines
#in a drawing area widget -- a widget in which I'm displaying a bitmap using
#XPutImage().  If doesn't matter if the lines I draw interactively stay
#around when the window is refreshed.
#
#Currently, to draw interactively, I begin with:
#
#   /* drawIndex is an colortable index I reserve for the Foreground */
#   /* my_default_bg_color is the color index for the background of my image */
#   palette_colors[drawIndex].red =   palette_colors[my_default_bg_color].red;
#   palette_colors[drawIndex].green = palette_colors[my_default_bg_color].green;
#   palette_colors[drawIndex].blue =  palette_colors[my_default_bg_color].blue;
#   XStoreColors( myDisplay, my_cmap, &palette_colors[DrawIndex], 1);
#   XFlush( myDisplay);
#
#   XSetFunction( myDisplay, gc, GXxor);
#  

In [50]:
print(base[list(orden[21])[0]])

I'm trying to write some code that lets me draw lines and do rubber-band
boxes in Motif/X.  I'm running on an 8-bit display for which I've created a
colormap and am using almost all of the colors.  I want to draw the lines
in a drawing area widget -- a widget in which I'm displaying a bitmap using
XPutImage().  If doesn't matter if the lines I draw interactively stay
around when the window is refreshed.

Currently, to draw interactively, I begin with:

   /* drawIndex is an colortable index I reserve for the Foreground */
   /* my_default_bg_color is the color index for the background of my image */
   palette_colors[drawIndex].red =   palette_colors[my_default_bg_color].red;
   palette_colors[drawIndex].green = palette_colors[my_default_bg_color].green;
   palette_colors[drawIndex].blue =  palette_colors[my_default_bg_color].blue;
   XStoreColors( myDisplay, my_cmap, &palette_colors[DrawIndex], 1);
   XFlush( myDisplay);

   XSetFunction( myDisplay, gc, GXxor);
   XSetForeground( myDi

## Ejercicio
+ Evalúa la búsqueda con distintos valores de $r$ y $\eta$ usando la fórmula de 
$$
  l = \frac{log(0.5)}{log(1 - \eta^r)}
$$

+ Verifica que las colisiones de los conjuntos aproximan la similitud de Jaccard.
+ Extiende la clase `MinHashTable` para que tome en cuenta las multiplicidades de las bolsas.